In [1]:
%pip install tensorflow
%pip install matplotlib
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 8.3/8.3 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Loading the data

In [1]:
import glob

data_dir = 'data/generated'
data_files = glob.glob(data_dir + "/*.csv")

In [2]:
import pandas as pd

df_list = []

for file in data_files:
    data_frame = pd.read_csv(file, index_col=None, header=0)
    df_list.append(data_frame)

df = pd.concat(df_list, ignore_index=True)

print(df.shape)
print(df.columns)

(995, 2)
Index(['pregunta', 'clase'], dtype='object')


In [3]:
df_test = pd.read_csv('data/preguntas_test.csv', index_col=None, header=0)
print(df_test.shape)
print(df.columns)

(19, 2)
Index(['pregunta', 'clase'], dtype='object')


In [4]:
NUMBER_OF_CLASSES = df['clase'].nunique()
MIN_NUMBER_OF_QUESTIONS = df[df['clase'] == df.groupby('clase').size().idxmin()].shape[0]

In [5]:
df_raw = df.groupby('clase').apply(lambda x: x.sample(n=MIN_NUMBER_OF_QUESTIONS)).reset_index(drop=True)
print(df_raw.shape)

(900, 2)


In [6]:
classes_list = df['clase'].unique().tolist()
print(classes_list)

['Definicion', 'EntidadAbreviatura', 'EntdidadGrupo', 'EntidadLugar', 'EntidadLugarCapital', 'EntidadLugarCiudad', 'EntidadLugarPais', 'EntidadObjeto', 'EntidadPersona', 'Evento', 'NumericoCantidad', 'NumericoEdad', 'NumericoMedida', 'NumericoPeriodo', 'NumericoPorcentaje', 'TemporalAnyo', 'TemporalFecha', 'TemporalMes']


In [7]:
classes_df_1 = classes_df = pd.DataFrame(columns=classes_list).reset_index(drop=True)
classes_df_2 = classes_df = pd.DataFrame(columns=classes_list).reset_index(drop=True)

for index, row in df_raw.iterrows():
    classes_df_1.loc[len(classes_df_1)] = [1 if row['clase'] == c else 0 for c in classes_list]

for index, row in df_test.iterrows():
    classes_df_2.loc[len(classes_df_2)] = [1 if row['clase'] == c else 0 for c in classes_list]

In [8]:
classes_df_1.sample(3)

,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
867,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
484,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
310,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [9]:
classes_df_2.sample(3)

,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [10]:
df_raw.sample(3)

,pregunta,clase
847,¿En qué fecha se conmemora el Día Internaciona...,TemporalFecha
859,¿Cuál es el mes de la celebración del Día de S...,TemporalMes
821,¿En qué día de la semana te gusta ir al cine?,TemporalFecha


In [11]:
df_test.sample(3)

,pregunta,clase
13,¿Qué es UNICEF?,Definicion
12,¿Qué es lo que causa el agujero de ozono?,EntidadObjeto
4,¿Qué cargo detenta Ariel Sharon?,EntidadObjeto


In [12]:
df_one_hot_encoded = pd.concat([df_raw, classes_df_1], axis=1).drop('clase', axis=1)
df_test_one_hot_encoded = pd.concat([df_test, classes_df_2], axis=1).drop('clase', axis=1)

In [13]:
df_one_hot_encoded.sample(3)

,pregunta,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
301,¿De qué nacionalidad es el director de cine Qu...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
236,¿Cuál es la capital de Qatar?,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
156,¿En dónde puedo encontrar la mejor oferta de r...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df_test_one_hot_encoded.sample(3)

,pregunta,Definicion,EntidadAbreviatura,EntdidadGrupo,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
7,¿Dónde tiene lugar el Motorshow?,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,¿Qué firma está acusada de explotación laboral...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
13,¿Qué es UNICEF?,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Preprocessing the data

In [15]:
df_one_hot_encoded['pregunta'] = df_raw['pregunta'].str.lower()
df_test_one_hot_encoded['pregunta'] = df_test['pregunta'].str.lower()

In [16]:
df_one_hot_encoded = df_one_hot_encoded.sample(frac=1)
df_test_one_hot_encoded = df_test_one_hot_encoded.sample(frac=1)

# Model creation

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, optimizers
import tensorflow as tf

In [19]:
SEED = 30072001
X_train, x_test, Y_train, y_test = train_test_split(df_one_hot_encoded['pregunta'], 
                                                    df_one_hot_encoded[df_raw.columns[1:]], 
                                                    test_size=0.1, 
                                                    random_state=SEED)

In [18]:
X_train = df_one_hot_encoded['pregunta']
Y_train = df_one_hot_encoded[df_one_hot_encoded.columns[1:]]

X_test = df_test_one_hot_encoded['pregunta']
Y_test = df_test_one_hot_encoded[df_test_one_hot_encoded.columns[1:]]

In [19]:
MAX_FEATURES = 150
SEQUENCE_LENGTH = 50

vectorize_layer = layers.TextVectorization(
    standardize=None, 
    max_tokens=MAX_FEATURES, 
    output_mode='int', 
    output_sequence_length=SEQUENCE_LENGTH)

vectorize_layer.adapt(X_train)

In [103]:
EMBEDDING_DIM = 8

model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(layers.Embedding(MAX_FEATURES + 1, EMBEDDING_DIM))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(75, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(NUMBER_OF_CLASSES, activation="sigmoid"))

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 50)               0         
 torization)                                                     
                                                                 
 embedding_16 (Embedding)    (None, 50, 8)             1208      
                                                                 
 dense_50 (Dense)            (None, 50, 100)           900       
                                                                 
 global_average_pooling1d_15  (None, 100)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_51 (Dense)            (None, 75)                7575      
                                                                 
 dropout_17 (Dropout)        (None, 75)              

In [104]:
model.compile(optimizer=optimizers.Adam(0.035), loss='binary_crossentropy', metrics=['accuracy' , tf.keras.metrics.AUC()])

In [105]:
history = model.fit(X_train, Y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 1s 14ms/step - loss: 0.3026 - accuracy: 0.0542 - auc_16: 0.4976 - val_loss: 0.2196 - val_accuracy: 0.0444 - val_auc_16: 0.4870
Epoch 2/10
23/23 [==============================] - 0s 5ms/step - loss: 0.2247 - accuracy: 0.0542 - auc_16: 0.4964 - val_loss: 0.2139 - val_accuracy: 0.0889 - val_auc_16: 0.5401
Epoch 3/10
23/23 [==============================] - 0s 5ms/step - loss: 0.2067 - accuracy: 0.0889 - auc_16: 0.6412 - val_loss: 0.1908 - val_accuracy: 0.1500 - val_auc_16: 0.7194
Epoch 4/10
23/23 [==============================] - 0s 5ms/step - loss: 0.1916 - accuracy: 0.1292 - auc_16: 0.7391 - val_loss: 0.1846 - val_accuracy: 0.1722 - val_auc_16: 0.7729
Epoch 5/10
23/23 [==============================] - 0s 5ms/step - loss: 0.1756 - accuracy: 0.2028 - auc_16: 0.8230 - val_loss: 0.1611 - val_accuracy: 0.3389 - val_auc_16: 0.8731
Epoch 6/10
23/23 [==============================] - 0s 5ms/step - loss: 0.1477 - accuracy: 0.3722 - auc_16: 0

In [106]:
metrics = model.evaluate(X_test, Y_test)
print("{}: {}".format(model.metrics_names[1], metrics[1]))

1/1 [==============================] - 0s 24ms/step - loss: 0.2606 - accuracy: 0.4211 - auc_16: 0.7479
accuracy: 0.42105263471603394


In [71]:
def interpret(r):
    max_prob = float('-inf')
    index = -1
    for i, prob in enumerate(r[0]):
        if (prob > max_prob):
            max_prob = prob
            index = i
    
    print("Clasificación: " + classes_list[index] + " con un " + max_prob.astype('str') + " de probabilidad.")



In [72]:
for q in X_test:
    print(q)
    interpret(model.predict([q]))
    print("\n")

¿cómo murió jimi hendrix?
1/1 [==============================] - 0s 127ms/step
Clasificación: Evento con un 0.9914384 de probabilidad.


¿qué es el mossad?
1/1 [==============================] - 0s 36ms/step
Clasificación: Definicion con un 0.8128443 de probabilidad.


¿qué año comenzó la intifada?
1/1 [==============================] - 0s 38ms/step
Clasificación: TemporalAnyo con un 0.83507425 de probabilidad.


¿en qué día cae el día del año nuevo chino?
1/1 [==============================] - 0s 37ms/step
Clasificación: TemporalFecha con un 0.66946465 de probabilidad.


¿qué es lo que causa el agujero de ozono?
1/1 [==============================] - 0s 37ms/step
Clasificación: Definicion con un 0.12101277 de probabilidad.


¿qué firma está acusada de explotación laboral infantil?
1/1 [==============================] - 0s 37ms/step
Clasificación: Definicion con un 0.8304003 de probabilidad.


¿qué es unicef?
1/1 [==============================] - 0s 35ms/step
Clasificación: Definicion